Read features.csv

In [1]:
import pandas as pd
from pathlib import Path

split_name = "train_test"
features_path = Path("resources") / split_name / "features.csv"

features_df = pd.read_csv(features_path)
features_df.head()

name   term freq  prediction_length     domain     trend   
0  loop_seattle  short   5T                 48  Transport  0.594711  \
1  loop_seattle  short    D                 30  Transport  0.500107   
2  loop_seattle  short    H                 48  Transport  0.446253   
3       m_dense  short    D                 30  Transport  0.729707   
4       m_dense  short    H                 48  Transport  0.397013   

   seasonal_strength   entropy     hurst  stability  lumpiness  
0           0.280343  0.589723  0.936951   0.458372   0.613632  
1           0.769271  0.576937  0.611051   0.170957   0.339067  
2           0.630205  0.656869  0.689547   0.211964   0.454267  
3           0.741856  0.681616  0.744518   0.353943   0.367117  
4           0.802983  0.477337  0.588340   0.113609   0.155365

Initialize a col for CRPS scores

In [2]:
import numpy as np

features_df["true_crps"] = np.nan
features_df.head()

name   term freq  prediction_length     domain     trend   
0  loop_seattle  short   5T                 48  Transport  0.594711  \
1  loop_seattle  short    D                 30  Transport  0.500107   
2  loop_seattle  short    H                 48  Transport  0.446253   
3       m_dense  short    D                 30  Transport  0.729707   
4       m_dense  short    H                 48  Transport  0.397013   

   seasonal_strength   entropy     hurst  stability  lumpiness  true_crps  
0           0.280343  0.589723  0.936951   0.458372   0.613632        NaN  
1           0.769271  0.576937  0.611051   0.170957   0.339067        NaN  
2           0.630205  0.656869  0.689547   0.211964   0.454267        NaN  
3           0.741856  0.681616  0.744518   0.353943   0.367117        NaN  
4           0.802983  0.477337  0.588340   0.113609   0.155365        NaN

Read CRPS scores

In [3]:
model_name = "chronos_bolt_base"
scores_path = Path("results") / model_name / "all_results.csv"
scores_df = pd.read_csv(scores_path)
scores_df.head()

dataset              model  eval_metrics/MSE[mean]   
0           ett1/W/short  chronos_bolt_base            1.637703e+06  \
1  bitbrains_rnd/H/short  chronos_bolt_base            1.862107e+06   
2     m4_monthly/M/short  chronos_bolt_base            1.934417e+06   
3        solar/10T/short  chronos_bolt_base            2.491767e+01   
4       solar/10T/medium  chronos_bolt_base            2.785010e+01   

   eval_metrics/MSE[0.5]  eval_metrics/MAE[0.5]  eval_metrics/MASE[0.5]   
0           1.637703e+06             984.215751                1.697033  \
1           1.862107e+06             167.787259                5.896931   
2           1.934417e+06             565.388997                0.948831   
3           2.491767e+01               2.240112                0.990670   
4           2.785010e+01               2.342676                1.026770   

   eval_metrics/MAPE[0.5]  eval_metrics/sMAPE[0.5]  eval_metrics/MSIS   
0                0.666664                 0.568513           9.146041  \
1                2.093076                 0.579010         204.743293   
2                0.159006                 0.131941           7.587628   
3                5.422044                 1.529881           7.271073   
4                3.019735                 1.508759          12.948194   

   eval_metrics/RMSE[mean]  eval_metrics/NRMSE[mean]  eval_metrics/ND[0.5]   
0              1279.727650                  0.509234              0.391643  \
1              1364.590466                  5.972116              0.734319   
2              1390.833069                  0.289063              0.117507   
3                 4.991760                  1.452102              0.651648   
4                 5.277320                  1.171058              0.519849   

   eval_metrics/mean_weighted_sum_quantile_loss        domain  num_variates  
0                                      0.295789        Energy             7  
1                                      0.624238  Web/CloudOps             2  
2                                      0.093805      Econ/Fin             1  
3                                      0.511279        Energy             1  
4                                      0.436094        Energy             1

Combine scores df with dataset info df

In [4]:
def get_pretty_name(name) -> str:
    """
    Returns the dataset's key for accessing dataset infomation in
    dataset_properties.json (e.g. domain and number of variates)

    Args:
        name (str): Name of the dataset.
    """
    pretty_names = {
        "saugeenday": "saugeen",
        "temperature_rain_with_missing": "temperature_rain",
        "kdd_cup_2018_with_missing": "kdd_cup_2018",
        "car_parts_with_missing": "car_parts",
    }
    return pretty_names.get(name, name)

for i, score_row in scores_df.iterrows():
    name, freq, term = score_row["dataset"].split("/")
    name = get_pretty_name(name.lower())
    freq = freq.split("-")[0]

    mask = (
        (features_df["name"].str.lower().apply(get_pretty_name) == name)
        & (features_df["freq"] == freq)
        & (features_df["term"] == term)
    )

    features_df.loc[mask, "true_crps"] = score_row[
        "eval_metrics/mean_weighted_sum_quantile_loss"
    ]


print(f"saving updated features to {features_path}")
features_df.head()
features_df.to_csv(features_path, index=False)

saving updated features to resources\train_test\features.csv
